In [1]:
import boto3
import json, uuid, copy, datetime
import random, names, tabulate
import pandas as pd

In [2]:
# open_cmd_in_new_terminal("docker-compose up")
!gnome-terminal -- docker-compose up

In [17]:
# !./setuprds.sh
!./setupdynamo.sh

Creating the DynamoDB table that holds the client floats
{
    "StackId": "arn:aws:cloudformation:us-east-1:123456789:stack/client-float-table/d81298b5-eb8b-49bc-b12d-3cf5d5f6b8ce"
}
Adding in item for core, ZAR wholesale float


In [4]:
!./deploylambdas.sh

Deploying the float lambda
Serverless: config.options_stage: local
Serverless: serverless.service.custom.stage: undefined
Serverless: serverless.service.provider.stage: dev
Serverless: config.stage: local
Serverless: Using serverless-localstack
Serverless: Reconfiguring service apigateway to use http://localhost:4567
Serverless: Reconfiguring service cloudformation to use http://localhost:4581
Serverless: Reconfiguring service cloudwatch to use http://localhost:4582
Serverless: Reconfiguring service lambda to use http://localhost:4574
Serverless: Reconfiguring service dynamodb to use http://localhost:4569
Serverless: Reconfiguring service kinesis to use http://localhost:4568
Serverless: Reconfiguring service route53 to use http://localhost:4580
Serverless: Reconfiguring service firehose to use http://localhost:4573
Serverless: Reconfiguring service stepfunctions to use http://localhost:4585
Serverless: Reconfiguring service es to use http://localhost:4578
Serverless: Reconfiguring serv

In [5]:
dynamodb = boto3.client('dynamodb', endpoint_url = 'http://localhost:4569')
local_lambda = boto3.client('lambda', endpoint_url = 'http://localhost:4574')

In [6]:
def print_deployed_functions():
    function_list = local_lambda.list_functions()
    # print(function_list)
    print('Function list: ', [function['FunctionName'] for function in function_list['Functions']])

In [7]:
def generate_account(client_id = 'zar_client_co', float_id = 'zar_cash_float'):
    first_name = names.get_first_name()
    family_name = names.get_last_name()
    user_id = str(uuid.uuid4())
    return { 'clientId': client_id, 'floatId': float_id, 'ownerUserId': user_id, 'userFirstName': first_name, 'userFamilyName': family_name}

In [8]:
def decode_lambda_result(lambda_result):
    lambda_payload = lambda_result['Payload'].read()
    lambda_pload_decoded = lambda_payload.decode('utf-8')
    lambda_pload_object = json.loads(lambda_pload_decoded)
    return lambda_pload_object

In [9]:
def create_number_accounts(number_accounts = 1, client_id = 'zar_client_co'):
    account_dicts = [generate_account(client_id) for i in range(number_accounts)]
    persisted_accounts = []
    for account in account_dicts:
        lambda_result = local_lambda.invoke(FunctionName='create-account', InvocationType='RequestResponse', 
                                           Payload=json.dumps(account))
        lambda_payload = json.loads(decode_lambda_result(lambda_result)['body'])
        persisted_account = copy.deepcopy(account)
        persisted_account['accountId'] = lambda_payload['accountId']
        persisted_account['persistedTime'] = lambda_payload['persistedTime']
        print('Account with ID %s persisted at %s' % (persisted_account['accountId'], persisted_account['persistedTime']))
        persisted_accounts.append(persisted_account)
    
    return persisted_accounts
    

In [10]:
def generate_saving_transaction(account_id, ref_amount = 100, float_id = 'zar_cash_primary'):
    current_time = datetime.datetime.now()
    saved_amount = round(random.random() * ref_amount * 10 * 10) # random proportion of ref amount, 
    test_saving_dict = { 
        'accountId': account_id, 
        'initiationTime': str(current_time), 
        'settlementTime': str(current_time), 
        'savedAmount': saved_amount, 
        'savedCurrency': 'ZAR', 
        'savedUnit': 'HUNDREDTH_CENT',
        'floatId': float_id
    }
    return test_saving_dict

In [11]:
def seed_savings_for_accounts(accounts, tx_per_account = 1, base_amount = 100):
    transactions = []
    for i in range(tx_per_account):
        transactions.extend([generate_saving_transaction(account['accountId'], base_amount) for account in accounts])
#     print('transactions: ', transactions)
    tx_records = []
    for tx in transactions:
        lambda_result = local_lambda.invoke(FunctionName='add-savings', InvocationType='RequestResponse', 
                                           Payload=json.dumps(tx, default=str))
        lambda_payload = json.loads(decode_lambda_result(lambda_result)['body'])
        persisted_tx = copy.deepcopy(tx)
        persisted_tx['transactionId'] = lambda_payload['transactionDetails'][0][0]['transaction_id']
        persisted_tx['currentBalance'] = lambda_payload['newBalance']
        # persisted_account['persistedTime'] = lambda_payload['persistedTime']
        print('Tx for account %s persisted with new balance %s' % (tx['accountId'], persisted_tx['currentBalance']))
        tx_records.append(persisted_tx)
    
    return tx_records

In [12]:
def run_accrual_for_float(client_id = 'zar_client_co', float_id = 'zar_cash_primary', accrued_amount = 100):
    accrualDict = {
        'clientId': client_id,
        'floatId': float_id,
        'accrualAmount': accrued_amount,
        'currency': 'ZAR',
        'unit': 'HUNDREDTH_CENT',
        'backingEntityIdentifier': 'tx-id-backing'
    }
    lambda_result = local_lambda.invoke(FunctionName='float-api-local-accrue', InvocationType='RequestResponse', 
                                               Payload=json.dumps(accrualDict, default=str))
    lambda_payload = json.loads(decode_lambda_result(lambda_result)['body'])
    return lambda_payload

In [13]:
print_deployed_functions()

Function list:  ['float-api-local-accrue', 'create-account', 'add-savings']


In [14]:
accounts = create_number_accounts(number_accounts = 10)

Account with ID debb3be9-2b9c-442b-adbd-e8dd526b9671 persisted at 2019-06-23T15:12:21.452Z
Account with ID 7faf4f5e-d0b0-473a-88d1-990f82f5d785 persisted at 2019-06-23T15:12:22.615Z
Account with ID b3b4ea2c-174d-4bcf-9be7-8314593deaeb persisted at 2019-06-23T15:12:23.246Z
Account with ID a261cc16-16c8-4642-bee6-de8d6430387f persisted at 2019-06-23T15:12:23.772Z
Account with ID 682eac07-51d0-4aa9-8091-46ac47a0c74c persisted at 2019-06-23T15:12:24.303Z
Account with ID c0a16079-f4f1-4826-a729-9e5c5ceab44e persisted at 2019-06-23T15:12:24.792Z
Account with ID f1542815-66f8-4590-b841-27f94c865948 persisted at 2019-06-23T15:12:25.297Z
Account with ID 477e6c3c-7ffc-4a3b-b256-ce7273d444db persisted at 2019-06-23T15:12:25.773Z
Account with ID 3457d8b1-2918-498a-a7d4-687815595c65 persisted at 2019-06-23T15:12:26.400Z
Account with ID ee3f4ab0-a3b3-4aef-9935-74ace7619c43 persisted at 2019-06-23T15:12:26.911Z


In [15]:
account_info = [{ 'Name': account['userFirstName'] + ' ' + account['userFamilyName'], 
                'AccountId': account['accountId'] } for account in accounts[:100]]
print(tabulate.tabulate(account_info, headers = 'keys'))

Name               AccountId
-----------------  ------------------------------------
Benny Steib        debb3be9-2b9c-442b-adbd-e8dd526b9671
Regina Smith       7faf4f5e-d0b0-473a-88d1-990f82f5d785
William Prattis    b3b4ea2c-174d-4bcf-9be7-8314593deaeb
Vicki Galvan       a261cc16-16c8-4642-bee6-de8d6430387f
Daniel Dickson     682eac07-51d0-4aa9-8091-46ac47a0c74c
Christopher Hardy  c0a16079-f4f1-4826-a729-9e5c5ceab44e
Bernice White      f1542815-66f8-4590-b841-27f94c865948
Wayne Malone       477e6c3c-7ffc-4a3b-b256-ce7273d444db
John Thill         3457d8b1-2918-498a-a7d4-687815595c65
Roy Rose           ee3f4ab0-a3b3-4aef-9935-74ace7619c43


In [18]:
single_tx = generate_saving_transaction(accounts[0]['accountId'], 100)
lambda_result = local_lambda.invoke(FunctionName='add-savings', Payload=json.dumps(single_tx, default=str))
result_decoded = decode_lambda_result(lambda_result)
print('Lambda result', result_decoded)

Lambda result {'body': '{"transactionDetails":[[{"transaction_id":"0b89a0af-979b-470e-a163-f9ed725b016b","creation_time":"2019-06-23T15:12:54.747Z"}],[{"transaction_id":"0fcb3090-1f9e-4e03-a351-4687cfcef3b5","creation_time":"2019-06-23T15:12:54.747Z"},{"transaction_id":"0fcb3090-1f9e-4e03-a351-4687cfcef3b5","creation_time":"2019-06-23T15:12:54.747Z"}]],"newBalance":11158}', 'statusCode': 200}


In [19]:
account_balances = seed_savings_for_accounts(accounts[:100], tx_per_account = 3, base_amount = 100 * 100 * 100)

Tx for account debb3be9-2b9c-442b-adbd-e8dd526b9671 persisted with new balance 35897950
Tx for account 7faf4f5e-d0b0-473a-88d1-990f82f5d785 persisted with new balance 3136636
Tx for account b3b4ea2c-174d-4bcf-9be7-8314593deaeb persisted with new balance 63960078
Tx for account a261cc16-16c8-4642-bee6-de8d6430387f persisted with new balance 3274044
Tx for account 682eac07-51d0-4aa9-8091-46ac47a0c74c persisted with new balance 67856730
Tx for account c0a16079-f4f1-4826-a729-9e5c5ceab44e persisted with new balance 81608100
Tx for account f1542815-66f8-4590-b841-27f94c865948 persisted with new balance 81544101
Tx for account 477e6c3c-7ffc-4a3b-b256-ce7273d444db persisted with new balance 2099594
Tx for account 3457d8b1-2918-498a-a7d4-687815595c65 persisted with new balance 93909036
Tx for account ee3f4ab0-a3b3-4aef-9935-74ace7619c43 persisted with new balance 25341274
Tx for account debb3be9-2b9c-442b-adbd-e8dd526b9671 persisted with new balance 122142271
Tx for account 7faf4f5e-d0b0-473a-

In [ ]:
def merge_dataframes(accounts, account_balances):
    df = pd.DataFrame(account_balances)
    df = df[df['currentBalance'] == df.groupby('accountId')['currentBalance'].transform('max')]
    df = df[['accountId', 'currentBalance']]
    df['currentBalance'] = pd.to_numeric(df['currentBalance'])
    df['balanceRand'] = df['currentBalance']/10000
    account_name_df = pd.DataFrame(accounts)[['accountId', 'floatId', 'userFamilyName', 'userFirstName']]
    account_name_df['Holder Name'] = account_name_df['userFirstName'] + ' ' + account_name_df['userFamilyName']
    account_name_df.set_index('accountId')
    df.set_index('accountId')
    merged_df = pd.merge(account_name_df, df)[['accountId', 'floatId', 'Holder Name', 'balanceRand']]
    return merged_df

In [ ]:
merged_df = merge_dataframes(accounts, account_balances)

In [ ]:
merged_df

In [20]:
amount_to_accrue = round(random.random() * 100 * 100 * 100)
print('Instructing accrual of : R', amount_to_accrue / (100 * 100))
accrual_result = run_accrual_for_float(accrued_amount = amount_to_accrue)
allocation_details = json.loads(accrual_result['userAllocationTransactions']['body'])
account_transaction_list = allocation_details['allocationRecords']['accountTxIds']
# print(tabulate.tabulate(account_transaction_list, headers = 'keys'))

Instructing accrual of : R 14.9734


In [21]:
# print(accrual_result)
account_transactions = allocation_details['allocationRecords']['accountTxIds']
number_of_float_transactions = len(allocation_details['allocationRecords']['floatTxIds'])
number_of_account_transactions = len(account_transactions)
print('%d float transactions, %d account transactions' % (number_of_float_transactions, number_of_account_transactions))
sum_of_accounts = sum([transaction['amount'] for transaction in account_transactions])
print('Sum of account additions: ', sum_of_accounts)
bonus_allocation = accrual_result['entityAllocations']['bonusShare']
company_allocation = accrual_result['entityAllocations']['clientShare']
print('Bonus allocation: %d, and client co allocation: %d' % (bonus_allocation, company_allocation))
total_allocations = sum_of_accounts + bonus_allocation + company_allocation
print('Of accrual %d, allocations %d' % (amount_to_accrue, total_allocations))
rounding_allocated_to_bonus = isinstance(allocation_details['bonusAllocation'], list)
print('Was there a rounding allocation? : ', rounding_allocated_to_bonus)
print('Rounding allocation record: ', allocation_details['bonusAllocation'])

31 float transactions, 31 account transactions
Sum of account additions:  119786
Bonus allocation: 14973, and client co allocation: 14973
Of accrual 149734, allocations 149732
Was there a rounding allocation? :  True
Rounding allocation record:  [{'BONUS': 'dcd9c5c4-3383-4cf3-aacb-f5a20a9d662c'}]
